In [203]:
import os
import json
import random
import sqlite3
import sqlparse
import pandas as pd
import numpy as np

import seaborn as sns
import mysql.connector

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt

In [204]:
import pandas as pd # Importing the pandas library
import numpy as np

# prompt: upload rthe csv

#from google.colab import files
#uploaded = files.upload()
# Assuming the uploaded file is named 'your_file.csv'
# Replace 'your_file.csv' with the actual file name if it's different
df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized
df.head()


C:\Users\bonas\AppData\Local\Temp\ipykernel_5600\974499630.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [205]:
# df = pd.read_csv('/kaggle/input/airbnbopendata/Airbnb_Open_Data.csv')

# *Renaming the columns*

In [206]:
#df.columns = df.columns.str.upper()

In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [208]:
df.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license'],
      dtype='object')

# *Deleting the reduntant columns*

In [209]:

columns_to_drop = ['license']

In [210]:
df.drop(columns=columns_to_drop,inplace=True)

In [211]:
#df.columns == columns_to_keep

In [212]:
#df.head()

# *Dropping the duplicates*

In [213]:
df.duplicated().sum()

541

In [214]:
df.drop_duplicates(inplace=True)

In [215]:
df.duplicated().sum()

0

# *Removing the NaN values*

In [216]:
df.isna().sum()

id                                    0
NAME                                250
host id                               0
host_identity_verified              289
host name                           404
neighbourhood group                  29
neighbourhood                        16
lat                                   8
long                                  8
country                             532
country code                        131
instant_bookable                    105
cancellation_policy                  76
room type                             0
Construction year                   214
price                               247
service fee                         273
minimum nights                      400
number of reviews                   183
last review                       15832
reviews per month                 15818
review rate number                  319
calculated host listings count      319
availability 365                    448
house_rules                       51842


In [217]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


In [218]:
df.dropna(inplace=True)

In [219]:
df

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."
5,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,verified,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,United States,...,$577,$115,3.0,74.0,6/22/2019,0.59,3.0,1.0,374.0,"No smoking, please, and no drugs."
7,1005202,BlissArtsSpace!,90821839709,unconfirmed,Emma,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,United States,...,"$1,060",$212,45.0,49.0,10/5/2017,0.40,5.0,1.0,219.0,House Guidelines for our BnB We are delighted ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102025,57349744,COZY Large Private APARTMENT in EAST VILLAGE,97186426683,verified,Luna,Manhattan,East Village,40.72588,-73.98598,United States,...,$878,$176,6.0,22.0,5/24/2019,0.87,2.0,1.0,15.0,"No Smoking, No Pets"
102028,57351400,"Gorgeous condo 20min from NYC,close to the air...",27226581142,unconfirmed,Arthur,Queens,Rego Park,40.72999,-73.86043,United States,...,$994,$199,2.0,67.0,7/1/2019,2.58,5.0,1.0,278.0,1. Be respectful of me and my home 2. No Pets ...
102029,57351953,Cozy room in a 2B with backyard access,5155155913,unconfirmed,Cristina,Queens,Astoria,40.76987,-73.92679,United States,...,$822,$164,1.0,9.0,4/25/2019,0.91,3.0,1.0,0.0,I have other roommates so be respectful of that.
102031,57353057,Cozy bedroom in williamsburg,83101851929,verified,Russell,Brooklyn,Williamsburg,40.70870,-73.96710,United States,...,"$1,078",$216,1.0,58.0,6/10/2019,2.08,5.0,2.0,120.0,- $150 Pet Fee - 14.45% Tax Will be adjusted i...


In [220]:
df.isna().sum()

id                                0
NAME                              0
host id                           0
host_identity_verified            0
host name                         0
neighbourhood group               0
neighbourhood                     0
lat                               0
long                              0
country                           0
country code                      0
instant_bookable                  0
cancellation_policy               0
room type                         0
Construction year                 0
price                             0
service fee                       0
minimum nights                    0
number of reviews                 0
last review                       0
reviews per month                 0
review rate number                0
calculated host listings count    0
availability 365                  0
house_rules                       0
dtype: int64

# *Cleaning individual columns*

In [221]:
df.head(3)

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39415 entries, 0 to 102040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39415 non-null  int64  
 1   NAME                            39415 non-null  object 
 2   host id                         39415 non-null  int64  
 3   host_identity_verified          39415 non-null  object 
 4   host name                       39415 non-null  object 
 5   neighbourhood group             39415 non-null  object 
 6   neighbourhood                   39415 non-null  object 
 7   lat                             39415 non-null  float64
 8   long                            39415 non-null  float64
 9   country                         39415 non-null  object 
 10  country code                    39415 non-null  object 
 11  instant_bookable                39415 non-null  object 
 12  cancellation_policy             3941

# SERVICE FEE

In [223]:
df['service fee']

0         $193 
1          $28 
4          $41 
5         $115 
7         $212 
          ...  
102025    $176 
102028    $199 
102029    $164 
102031    $216 
102040    $196 
Name: service fee, Length: 39415, dtype: object

In [224]:
df['service fee'].unique()

array(['$193 ', '$28 ', '$41 ', '$115 ', '$212 ', '$204 ', '$58 ',
       '$121 ', '$219 ', '$74 ', '$171 ', '$118 ', '$168 ', '$106 ',
       '$13 ', '$152 ', '$90 ', '$186 ', '$66 ', '$19 ', '$227 ', '$32 ',
       '$189 ', '$101 ', '$77 ', '$231 ', '$133 ', '$35 ', '$82 ',
       '$120 ', '$21 ', '$145 ', '$65 ', '$89 ', '$218 ', '$163 ', '$22 ',
       '$182 ', '$165 ', '$37 ', '$17 ', '$48 ', '$11 ', '$131 ', '$102 ',
       '$240 ', '$119 ', '$215 ', '$191 ', '$16 ', '$233 ', '$170 ',
       '$201 ', '$109 ', '$105 ', '$123 ', '$12 ', '$238 ', '$157 ',
       '$198 ', '$206 ', '$122 ', '$208 ', '$140 ', '$10 ', '$20 ',
       '$14 ', '$110 ', '$88 ', '$98 ', '$234 ', '$151 ', '$172 ',
       '$107 ', '$138 ', '$194 ', '$139 ', '$117 ', '$31 ', '$26 ',
       '$156 ', '$124 ', '$162 ', '$93 ', '$51 ', '$239 ', '$161 ',
       '$149 ', '$50 ', '$144 ', '$225 ', '$45 ', '$73 ', '$33 ', '$211 ',
       '$167 ', '$236 ', '$185 ', '$83 ', '$30 ', '$36 ', '$183 ',
       '$111 ', '$142 

In [225]:
df['service fee'] = df['service fee'].str.split('$').str[1]
df['service fee']

0         193 
1          28 
4          41 
5         115 
7         212 
          ... 
102025    176 
102028    199 
102029    164 
102031    216 
102040    196 
Name: service fee, Length: 39415, dtype: object

In [226]:
#df['service fee'] = df['service fee'].astype(int)

In [227]:
#df.info()

# PRICE

In [228]:
df['price']

0           $966 
1           $142 
4           $204 
5           $577 
7         $1,060 
           ...   
102025      $878 
102028      $994 
102029      $822 
102031    $1,078 
102040      $982 
Name: price, Length: 39415, dtype: object

In [229]:
df['price'].unique()

array(['$966 ', '$142 ', '$204 ', ..., '$458 ', '$743 ', '$942 '],
      dtype=object)

In [230]:
df['price'] = df['price'].str.split('$').str[1].str.replace(',','')
df['price']

0          966 
1          142 
4          204 
5          577 
7         1060 
          ...  
102025     878 
102028     994 
102029     822 
102031    1078 
102040     982 
Name: price, Length: 39415, dtype: object

In [231]:
#df['price'] = df['price'].astype(int)

In [232]:
#df.info()

# ROOM TYPE

In [233]:
df['room type']

0            Private room
1         Entire home/apt
4         Entire home/apt
5         Entire home/apt
7            Private room
               ...       
102025    Entire home/apt
102028    Entire home/apt
102029       Private room
102031       Private room
102040       Private room
Name: room type, Length: 39415, dtype: object

In [234]:
df['room type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [235]:
from sklearn.preprocessing import OneHotEncoder

#encoder = OneHotEncoder()

In [236]:
#encoded = encoder.fit_transform(df[['ROOM TYPE']]).toarray()

In [237]:
#df_room_type = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [238]:
#df_room_type

In [239]:
#df = pd.concat([df,df_room_type],axis=1)

In [240]:
#df.drop(columns=['ROOM TYPE'],inplace=True)

In [241]:
#df.head(3)

In [242]:
#df.info()

# CANCELLATION_POLICY

In [243]:
df['cancellation_policy']

0           strict
1         moderate
4         moderate
5         flexible
7         moderate
            ...   
102025      strict
102028    moderate
102029    flexible
102031    flexible
102040      strict
Name: cancellation_policy, Length: 39415, dtype: object

In [244]:
df['cancellation_policy'].unique()

array(['strict', 'moderate', 'flexible'], dtype=object)

In [245]:
#df['cancellation_policy'].isna().sum()

In [246]:
#df[df['cancellation_policy'].isna()]

In [247]:
#df.dropna(inplace=True)

In [248]:
#encoded = encoder.fit_transform(df[['CANCELLATION_POLICY']]).toarray()

In [249]:
#encoded

In [250]:
#df_can_policy = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [251]:
#df_can_policy

In [252]:
#df = pd.concat([df,df_can_policy],axis=True)

In [253]:
#df.drop(columns=['CANCELLATION_POLICY'],inplace=True)

In [254]:
#df.info()

# INSTANT_BOOKABLE

In [255]:
df['instant_bookable']

0         False
1         False
4         False
5          True
7         False
          ...  
102025    False
102028    False
102029    False
102031     True
102040     True
Name: instant_bookable, Length: 39415, dtype: object

In [256]:
df['instant_bookable'].unique()

array([False, True], dtype=object)

In [257]:
#df.dropna(inplace=True)

In [258]:
#df['instant_bookable'].unique()

In [259]:
#df['instant_bookable'].isna().sum()

In [260]:
#df['instant_bookable'] = df['instant_bookable'].apply(lambda x:1 if x==True else 0)

In [261]:
df['instant_bookable'].value_counts()

instant_bookable
False    19793
True     19622
Name: count, dtype: int64

In [262]:
#df.info()

# COUNTRY & COUNTRY CODE

In [263]:
df['country code']

0         US
1         US
4         US
5         US
7         US
          ..
102025    US
102028    US
102029    US
102031    US
102040    US
Name: country code, Length: 39415, dtype: object

In [264]:
df['country code'].value_counts()

country code
US    39415
Name: count, dtype: int64

In [265]:
df['country'].value_counts()

country
United States    39415
Name: count, dtype: int64

In [266]:
#df['country'] = 1
#df['country code'] = 1

# NEIGHBOURHOOD

In [267]:
df['neighbourhood']

0                 Kensington
1                    Midtown
4                East Harlem
5                Murray Hill
7         Bedford-Stuyvesant
                 ...        
102025          East Village
102028             Rego Park
102029               Astoria
102031          Williamsburg
102040         East Elmhurst
Name: neighbourhood, Length: 39415, dtype: object

In [268]:
df['neighbourhood'].unique()

array(['Kensington', 'Midtown', 'East Harlem', 'Murray Hill',
       'Bedford-Stuyvesant', "Hell's Kitchen", 'Upper West Side',
       'Bushwick', 'South Slope', 'Williamsburg', 'Harlem', 'Gowanus',
       'Clinton Hill', 'East Village', 'Flatlands', 'Fort Greene',
       'West Village', 'Prospect Heights', 'Financial District',
       'Ridgewood', 'Park Slope', 'Flatiron District', 'Windsor Terrace',
       'Roosevelt Island', 'Lower East Side', 'Greenpoint', 'Chinatown',
       'Chelsea', 'Carroll Gardens', 'Inwood', 'Kingsbridge', 'Astoria',
       'Boerum Hill', 'Washington Heights', 'Rockaway Beach',
       'Crown Heights', 'Upper East Side', 'Woodlawn', 'Sunnyside',
       'Gramercy', 'East New York', 'Prospect-Lefferts Gardens',
       'Sheepshead Bay', 'Theater District', 'Emerson Hill',
       'Long Island City', 'Fort Hamilton', 'Bensonhurst', 'Sunset Park',
       'Concourse', 'Kips Bay', 'Brighton Beach', 'St. Albans',
       'Arrochar', 'East Flatbush', 'Nolita', 'Wakefiel

In [269]:
#df['neighbourhood'].isna().sum()

In [270]:
from sklearn.preprocessing import LabelEncoder

#label = LabelEncoder()

In [271]:
#encoded = label.fit_transform(df[['NEIGHBOURHOOD']])

In [272]:
#encoded

In [273]:
'''df_neighbourhood = pd.DataFrame({
    'NEIGHBOURHOOD_ENCODED' : encoded
})'''

"df_neighbourhood = pd.DataFrame({\n    'NEIGHBOURHOOD_ENCODED' : encoded\n})"

In [274]:
#df_neighbourhood

In [275]:
#df = pd.concat([df,df_neighbourhood],axis=1)

In [276]:
#df.drop(columns=['NEIGHBOURHOOD'],inplace=True)

In [277]:
#df.dropna(inplace=True)

In [278]:
#df.info()

# NEIGHBOURHOOD GROUP

In [279]:
df['neighbourhood group']

0          Brooklyn
1         Manhattan
4         Manhattan
5         Manhattan
7          Brooklyn
            ...    
102025    Manhattan
102028       Queens
102029       Queens
102031     Brooklyn
102040       Queens
Name: neighbourhood group, Length: 39415, dtype: object

In [280]:
df['neighbourhood group'].isna().sum()

0

In [281]:
df['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

In [282]:
#encoded = label.fit_transform(df[['NEIGHBOURHOOD GROUP']])

In [283]:
#encoded

In [284]:
''' = pd.DataFrame({
    'NEIGHBOURHOOD GROUP_ENCODED' : encoded
})'''

" = pd.DataFrame({\n    'NEIGHBOURHOOD GROUP_ENCODED' : encoded\n})"

In [285]:
#df_neighbourhood_group

In [286]:
#df = pd.concat([df,df_neighbourhood_group],axis=1)

In [287]:
#df.drop(columns=['NEIGHBOURHOOD GROUP'],inplace=True)

In [288]:
#df.dropna(inplace=True)

In [289]:
#df.info()

In [290]:
df['host name']

0         Madaline
1            Jenna
4           Lyndon
5         Michelle
7             Emma
            ...   
102025        Luna
102028      Arthur
102029    Cristina
102031     Russell
102040       Sonia
Name: host name, Length: 39415, dtype: object

In [291]:
df['host name'].unique()

array(['Madaline', 'Jenna', 'Lyndon', ..., 'Minh', 'Dier', 'Apostle John'],
      dtype=object)

In [292]:
#encoded = label.fit_transform(df[['HOST NAME']])

In [293]:
#encoded

In [294]:
#df_hostname = pd.DataFrame({
#})

In [295]:
#df_hostname

In [296]:
#df = pd.concat([df,df_hostname],axis=1)

In [297]:
#df.drop(columns=['HOST NAME'],inplace=True)

In [298]:
#df.dropna(inplace=True)

In [299]:
#df.head()

In [300]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39415 entries, 0 to 102040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39415 non-null  int64  
 1   NAME                            39415 non-null  object 
 2   host id                         39415 non-null  int64  
 3   host_identity_verified          39415 non-null  object 
 4   host name                       39415 non-null  object 
 5   neighbourhood group             39415 non-null  object 
 6   neighbourhood                   39415 non-null  object 
 7   lat                             39415 non-null  float64
 8   long                            39415 non-null  float64
 9   country                         39415 non-null  object 
 10  country code                    39415 non-null  object 
 11  instant_bookable                39415 non-null  object 
 12  cancellation_policy             3941

# HOST_IDENTITY_VERIFIED

In [301]:
df['host_identity_verified']

0         unconfirmed
1            verified
4            verified
5            verified
7         unconfirmed
             ...     
102025       verified
102028    unconfirmed
102029    unconfirmed
102031       verified
102040       verified
Name: host_identity_verified, Length: 39415, dtype: object

In [302]:
df['host_identity_verified'].unique()

array(['unconfirmed', 'verified'], dtype=object)

In [303]:
#df['host_identity_verified'].isna().sum()

In [304]:
#df['host_identity_verified'] = df['host_identity_verified'].apply(lambda x:1 if x=='verified' else 0)

In [305]:
df['host_identity_verified'].value_counts()

host_identity_verified
verified       19833
unconfirmed    19582
Name: count, dtype: int64

In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39415 entries, 0 to 102040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39415 non-null  int64  
 1   NAME                            39415 non-null  object 
 2   host id                         39415 non-null  int64  
 3   host_identity_verified          39415 non-null  object 
 4   host name                       39415 non-null  object 
 5   neighbourhood group             39415 non-null  object 
 6   neighbourhood                   39415 non-null  object 
 7   lat                             39415 non-null  float64
 8   long                            39415 non-null  float64
 9   country                         39415 non-null  object 
 10  country code                    39415 non-null  object 
 11  instant_bookable                39415 non-null  object 
 12  cancellation_policy             3941

# NAME

In [307]:
df['NAME']

0                        Clean & quiet apt home by the park
1                                     Skylit Midtown Castle
4          Entire Apt: Spacious Studio/Loft by central park
5                 Large Cozy 1 BR Apartment In Midtown East
7                                           BlissArtsSpace!
                                ...                        
102025         COZY Large Private APARTMENT in EAST VILLAGE
102028    Gorgeous condo 20min from NYC,close to the air...
102029               Cozy room in a 2B with backyard access
102031                         Cozy bedroom in williamsburg
102040                        Room in Queens, NY, near LGA.
Name: NAME, Length: 39415, dtype: object

In [308]:
df['NAME'].unique()

array(['Clean & quiet apt home by the park', 'Skylit Midtown Castle',
       'Entire Apt: Spacious Studio/Loft by central park', ...,
       'Beautiful Duplex & Private Terrace in Williamsburg',
       'Cozy room by South Seaport', 'Elmhurst 1st Fl Rightl Bedroom'],
      dtype=object)

In [309]:
df['NAME'].value_counts()

NAME
Private Room                                      12
Home away from home                               12
Cozy Apartment                                     9
Cozy Brooklyn Apartment                            9
20-25 minutes to downtown NYC/30-40 to midtown     9
                                                  ..
large house with inground pool/spa                 1
Classic Harlem --- Economy Room                    1
Large Beautiful BedStuy Apartment                  1
Private room in middle of Bushwick                 1
Elmhurst 1st Fl Rightl Bedroom                     1
Name: count, Length: 24773, dtype: int64

In [310]:
#encoded = label.fit_transform(df[['NAME']])

In [311]:
#encoded

In [312]:
#df_name = pd.DataFrame({
#    'NAME_ENCODED' : encoded
#})

In [313]:
#df_name

In [314]:
#df = pd.concat([df,df_name],axis=1)

In [315]:
#df.drop(columns=['NAME'],inplace=True)

In [316]:
#df.dropna(inplace=True)

In [317]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39415 entries, 0 to 102040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39415 non-null  int64  
 1   NAME                            39415 non-null  object 
 2   host id                         39415 non-null  int64  
 3   host_identity_verified          39415 non-null  object 
 4   host name                       39415 non-null  object 
 5   neighbourhood group             39415 non-null  object 
 6   neighbourhood                   39415 non-null  object 
 7   lat                             39415 non-null  float64
 8   long                            39415 non-null  float64
 9   country                         39415 non-null  object 
 10  country code                    39415 non-null  object 
 11  instant_bookable                39415 non-null  object 
 12  cancellation_policy             3941

In [318]:
#df.duplicated().sum()

In [319]:
df

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,966,193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,142,28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,204,41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."
5,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,verified,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,United States,...,577,115,3.0,74.0,6/22/2019,0.59,3.0,1.0,374.0,"No smoking, please, and no drugs."
7,1005202,BlissArtsSpace!,90821839709,unconfirmed,Emma,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,United States,...,1060,212,45.0,49.0,10/5/2017,0.40,5.0,1.0,219.0,House Guidelines for our BnB We are delighted ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102025,57349744,COZY Large Private APARTMENT in EAST VILLAGE,97186426683,verified,Luna,Manhattan,East Village,40.72588,-73.98598,United States,...,878,176,6.0,22.0,5/24/2019,0.87,2.0,1.0,15.0,"No Smoking, No Pets"
102028,57351400,"Gorgeous condo 20min from NYC,close to the air...",27226581142,unconfirmed,Arthur,Queens,Rego Park,40.72999,-73.86043,United States,...,994,199,2.0,67.0,7/1/2019,2.58,5.0,1.0,278.0,1. Be respectful of me and my home 2. No Pets ...
102029,57351953,Cozy room in a 2B with backyard access,5155155913,unconfirmed,Cristina,Queens,Astoria,40.76987,-73.92679,United States,...,822,164,1.0,9.0,4/25/2019,0.91,3.0,1.0,0.0,I have other roommates so be respectful of that.
102031,57353057,Cozy bedroom in williamsburg,83101851929,verified,Russell,Brooklyn,Williamsburg,40.70870,-73.96710,United States,...,1078,216,1.0,58.0,6/10/2019,2.08,5.0,2.0,120.0,- $150 Pet Fee - 14.45% Tax Will be adjusted i...


In [200]:
# prompt: create a new csv file with updated datatha has new data

# Assuming df is the DataFrame from the provided code
df.to_csv('updated_airbnb_data.csv', index=False)


# CREATING THE airbnb_nyc SCHEMA

In [121]:
# Step 1: Connect without specifying the database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Bonakele.1"
)
cursor = conn.cursor()

# Step 2: Create the database if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS airbnb_nyc")
cursor.execute("USE airbnb_nyc")

# CREATE TABLES IN SCHEMA
- Come up with table names
- Insert values for tables with unique and few values 
- Upload cleaned airbnb_nyc for table seperation
- Standardize column names

In [ ]:
# Neighbourhood Groups
cursor.execute("""
CREATE TABLE IF NOT EXISTS neighbourhood_groups (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50) NOT NULL UNIQUE
)
""")

# Neighbourhoods
cursor.execute("""
CREATE TABLE IF NOT EXISTS neighbourhoods (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    neighbourhood_group_id INT NOT NULL,
    UNIQUE (name, neighbourhood_group_id),
    FOREIGN KEY (neighbourhood_group_id) REFERENCES neighbourhood_groups(id)
)
""")

# Room Types
cursor.execute("""
CREATE TABLE IF NOT EXISTS room_types (
    id INT AUTO_INCREMENT PRIMARY KEY,
    type VARCHAR(50) NOT NULL UNIQUE
)
""")

# Cancellation Policies
cursor.execute("""
CREATE TABLE IF NOT EXISTS cancellation_policies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    policy VARCHAR(50) NOT NULL UNIQUE
)
""")

# Hosts
cursor.execute("""
CREATE TABLE IF NOT EXISTS hosts (
    id BIGINT PRIMARY KEY,
    name VARCHAR(100),
    identity_verified VARCHAR(20)
)
""")

# Listings
cursor.execute("""
CREATE TABLE IF NOT EXISTS listings (
    id BIGINT PRIMARY KEY,
    name VARCHAR(255),
    host_id BIGINT NOT NULL,
    neighbourhood_id INT,
    latitude DECIMAL(10, 8),
    longitude DECIMAL(11, 8),
    country VARCHAR(50),
    country_code VARCHAR(10),
    room_type_id INT,
    construction_year INT,
    price DECIMAL(10, 2),
    service_fee DECIMAL(10, 2),
    minimum_nights INT,
    number_of_reviews INT,
    last_review DATE,
    reviews_per_month DECIMAL(5, 2),
    review_rate_number INT,
    calculated_host_listings_count INT,
    availability_365 INT,
    instant_bookable BOOLEAN,
    cancellation_policy_id INT,
    house_rules TEXT,
    FOREIGN KEY (host_id) REFERENCES hosts(id),
    FOREIGN KEY (neighbourhood_id) REFERENCES neighbourhoods(id),
    FOREIGN KEY (room_type_id) REFERENCES room_types(id),
    FOREIGN KEY (cancellation_policy_id) REFERENCES cancellation_policies(id)
)
""")

cursor.execute("INSERT IGNORE INTO neighbourhood_groups (name) VALUES ('Brooklyn'), ('Manhattan'), ('Queens'), ('Bronx'), ('Staten Island')")
cursor.execute("INSERT IGNORE INTO room_types (type) VALUES ('Private room'), ('Entire home/apt'), ('Shared room')")
cursor.execute("INSERT IGNORE INTO cancellation_policies (policy) VALUES ('strict'), ('moderate'), ('flexible')")
conn.commit()

df = pd.read_csv("updated_airbnb_data.csv")

# Standardize column names
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# INSERT VALUES FOR TABLES WITH A LOT OF ROWS USING DF

In [126]:
# Insert unique neighbourhoods
neighbourhoods = df[['neighbourhood', 'neighbourhood_group']].drop_duplicates()

for _, row in neighbourhoods.iterrows():
    group_name = row['neighbourhood_group']
    if pd.notnull(group_name):  # Skip if it's NaN
        group_name = str(group_name)  # Convert to string just in case
        cursor.execute("SELECT id FROM neighbourhood_groups WHERE name = %s", (group_name,))
        group_id = cursor.fetchone()
        if group_id:
            # Now insert into neighbourhoods table
            cursor.execute("""
                INSERT IGNORE INTO neighbourhoods (name, neighbourhood_group_id)
                VALUES (%s, %s)
            """, (row['neighbourhood'], group_id[0]))

# Insert unique hosts  

hosts = df[['host_id', 'host_name', 'host_identity_verified']].drop_duplicates()

# Ensure types are correct
hosts['host_id'] = hosts['host_id'].astype(int)
hosts['host_name'] = hosts['host_name'].astype(str)
hosts['host_identity_verified'] = hosts['host_identity_verified'].astype(str)

# Now insert
for _, row in hosts.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO hosts (id, name, identity_verified)
        VALUES (%s, %s, %s)
    """, (row['host_id'], row['host_name'], row['host_identity_verified']))



# Insert unique listings
df_clean = df.dropna(subset=['neighbourhood', 'room_type', 'cancellation_policy', 'host_id'])

# Ensure all lookup columns are the correct types
df_clean['neighbourhood'] = df_clean['neighbourhood'].astype(str)
df_clean['room_type'] = df_clean['room_type'].astype(str)
df_clean['cancellation_policy'] = df_clean['cancellation_policy'].astype(str)
df_clean['host_id'] = df_clean['host_id'].astype(int)

# Then insert listings (or perform the lookup)
for _, row in df_clean.iterrows():
    cursor.execute("SELECT id FROM neighbourhoods WHERE name = %s", (row['neighbourhood'],))
    neighbourhood_id = cursor.fetchone()

    cursor.execute("SELECT id FROM room_types WHERE type = %s", (row['room_type'],))
    room_type_id = cursor.fetchone()

    cursor.execute("SELECT id FROM cancellation_policies WHERE policy = %s", (row['cancellation_policy'],))
    cancellation_policy_id = cursor.fetchone()
    
    # Insert listing
    cursor.execute("""
        INSERT IGNORE INTO listings (
            id, name, host_id, neighbourhood_id, latitude, longitude, country,
            country_code, room_type_id, construction_year, price, service_fee,
            minimum_nights, number_of_reviews, last_review, reviews_per_month,
            review_rate_number, calculated_host_listings_count, availability_365,
            instant_bookable, cancellation_policy_id, house_rules
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['id'], row['name'], row['host_id'], neighbourhood_id[0] if neighbourhood_id else None,
        row['lat'], row['long'], row['country'], row['country_code'], room_type_id[0] if room_type_id else None,
        row['construction_year'], row['price'], row['service_fee'],
        row['minimum_nights'], row['number_of_reviews'], row['last_review'],
        row['reviews_per_month'], row['review_rate_number'],
        row['calculated_host_listings_count'], row['availability_365'],
        row['instant_bookable'], cancellation_policy_id[0] if cancellation_policy_id else None,
        row['house_rules']
    ))

    conn.commit()

In [ ]:
# run this on anaconda prompt
## mysqldump -u root -p airbnb_nyc > airbnb_nyc.sql
# If this do not work run this To save a sql file:
## "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysqldump.exe" -u root -p airbnb_nyc > airbnb_nyc.sql

# CREATE DATABASE FILE

In [127]:
# Connect to MySQL
mysql_conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Bonakele.1",
    database="airbnb_nyc"
)
mysql_cursor = mysql_conn.cursor()

# Connect to SQLite (creates airbnb_nyc.db if it doesn't exist)
sqlite_conn = sqlite3.connect("airbnb_nyc.db")

# List of table names to transfer
tables = [
    "neighbourhood_groups",
    "neighbourhoods",
    "room_types",
    "cancellation_policies",
    "hosts",
    "listings"
]

# Transfer each table from MySQL to SQLite
for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", mysql_conn)
    df.to_sql(table, sqlite_conn, if_exists="replace", index=False)
    print(f"Transferred table: {table}")

# Close connections
mysql_cursor.close()
mysql_conn.close()
sqlite_conn.close()

C:\Users\bonas\AppData\Local\Temp\ipykernel_10956\3943947059.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", mysql_conn)


Transferred table: neighbourhood_groups
Transferred table: neighbourhoods
Transferred table: room_types
Transferred table: cancellation_policies
Transferred table: hosts
Transferred table: listings


In [328]:
# Load your database and create a database connection.
# You can connect to the sql database in any way you wish. 
# Use this method if you are unsure how to proceed. 
# Ensure the bike_store.db file is in the same directory as this notebook.
try:
    with sqlite3.connect("airbnb_nyc.db") as conn:
        print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

except sqlite3.OperationalError as e:
    print("Failed to open database:", e)

# List all tables in the database
pd.read_sql('''SELECT name FROM sqlite_master WHERE type='table';''',conn)     #Does the same thing

Opened SQLite database with version 3.49.1 successfully.


,name
0,neighbourhood_groups
1,neighbourhoods
2,room_types
3,cancellation_policies
4,hosts
5,listings
